In [5]:
import re
import string
import sys
import os
from bs4 import BeautifulSoup
import bs4
import requests as rq
import time
import pandas as pd

In [6]:
workroot="F:\\pythonfile\\Social Network\\crawler\\"
f = open(workroot+'pagesWithUid.txt', 'r')
pagesWithUid1 = []
allUids = []
for item in f:
    t = item.split('\t')
    t[1] = int(t[1])
    t = tuple(t)
    pagesWithUid1.append(t)
    allUids.append(t[0])

设置cookie

In [7]:
cookiessss={"Cookie":"_T_WM=09695a542dacbc5823bee01b2f582d8c; ALF=1499854684; SCF=AoHgqUpt5x21CVKVTyRY4ZThVaWp-7nQM8w1VlURgVup-gyZmmWh0VKtrGDVcyB_UmHZKDCGztGm8LVSC9yOxYE.; SUB=_2A250OhoNDeRhGeRM7VET-S7Nwj6IHXVXxKZFrDV6PUJbktBeLVTckW0IYuFl13bwFyccgmPqgPJ93L1ARw..; SUBP=0033WrSXqPxfM725Ws9jqgMF55529P9D9Wh43I5I.3h.v8H3jUH-sGWa5JpX5o2p5NHD95QEeoq0eo.7eK.EWs4DqcjB-cvVdJHXIPSXTP9DdcpLU5tt; SUHB=0YhQZqXFwu6eQL; SSOLoginState=1497262685"}

# 爬取用户的profile页

In [8]:
userinfo=dict()

In [9]:
counter=0
for uid in allUids:
    if uid not in userinfo:
        try:
            counter=counter+1
            url="https://weibo.cn/"+str(uid)+"/info"
            s=rq.Session()
            react=s.get(url,cookies=cookiessss)
            react.encoding='utf-8'
            soup=BeautifulSoup(react.text)
            tiplist=soup.find_all('div',class_="tip",string=re.compile("基本信息"))
            basicinfo=tiplist[0].next_sibling.contents
            basicinfo=[x for x in basicinfo if type(x)==bs4.element.NavigableString]
            infodic=dict()
            for x in basicinfo:
                x=str(x)
                for i in range(len(x)):
                    if x[i] in [":","："]:
                        break
                if i<len(x):
                    infodic[x[:i]]=x[i+1:]
            userinfo[uid]=infodic
            print(counter,'successful')
        except Exception as e:
            print('warning',counter,uid,e)
        #if counter>10:
        #    break
    else:
        counter=counter+1
        print(counter,uid,'is skipped')

C:\Users\哲超\Anaconda3\lib\site-packages\bs4\__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


1 successful
2 successful
3 successful
4 successful
5 successful
6 successful
7 successful
8 successful


KeyboardInterrupt: 

In [10]:
infodic

{'': '',
 '地区': '北京',
 '性别': '男',
 '昵称': '微软大数据',
 '标签': '',
 '生日': '0001-00-00',
 '简介': '在这里，让我们一起以全新视角审视与分析大数据时代下的企业发展，了解数据管理、分析和应用领域的最新业界资讯和技术。',
 '认证': '微软 SQL Server 官方微博',
 '认证信息': '微软 SQL Server 官方微博'}

In [11]:
pd.DataFrame(userinfo).T.to_csv(workroot+"微博profile.csv")

# 爬粉丝数、关注数等

In [12]:
user_inout=dict()

In [13]:
counter=0

for uid in allUids:
    if uid not in user_inout:
        try:
            counter=counter+1
            url="https://weibo.cn/"+str(uid)
            s=rq.Session()
            react=s.get(url,cookies=cookiessss)
            react.encoding='utf-8'
            soup=BeautifulSoup(react.text)
            guanzhu=soup.find("a",string=re.compile("关注\[\d*\]"))
            fenci=soup.find("a",string=re.compile("粉丝\[\d*\]"))
            templist=[guanzhu.string[3:-1],fenci.string[3:-1]]
            user_inout[uid]=templist
            print(counter,'successful')
        except Exception as e:
            print('warning',counter,uid,e)
        #if counter>10:
        #    break
    else:
        counter=counter+1
        print(counter,uid,'is skipped')

C:\Users\哲超\Anaconda3\lib\site-packages\bs4\__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


1 successful
2 successful
3 successful
4 successful


KeyboardInterrupt: 

In [ ]:
pd.DataFrame(user_inout).T.to_csv(workroot+"关注和粉丝.csv")

In [78]:
uid

'2685713683'

# 把地理分布信息输出到JSON格式，方便用Echarts作图

In [19]:
district=pd.read_excel(workroot+"微博profile.xlsx")

In [20]:
district["省份"]=district["地区"].apply(lambda x:x[:2])

In [21]:
to_Json=district.groupby("省份").count()["昵称"]

In [22]:
print(to_Json.to_json())

{"\u4e0a\u6d77":131,"\u4e91\u5357":1,"\u5176\u4ed6":10,"\u5317\u4eac":515,"\u5409\u6797":4,"\u56db\u5ddd":13,"\u5929\u6d25":6,"\u5b81\u590f":1,"\u5b89\u5fbd":5,"\u5c71\u4e1c":8,"\u5c71\u897f":2,"\u5e7f\u4e1c":107,"\u65b0\u7586":1,"\u6c5f\u82cf":23,"\u6cb3\u5317":2,"\u6cb3\u5357":4,"\u6d59\u6c5f":64,"\u6d77\u5357":1,"\u6d77\u5916":45,"\u6e56\u5317":9,"\u6e56\u5357":2,"\u798f\u5efa":15,"\u8fbd\u5b81":8,"\u91cd\u5e86":3,"\u9655\u897f":8,"\u9999\u6e2f":8,"\u9ed1\u9f99":3}


In [23]:
to_Json.sort_values()

省份
云南      1
海南      1
新疆      1
宁夏      1
山西      2
河北      2
湖南      2
重庆      3
黑龙      3
吉林      4
河南      4
安徽      5
天津      6
山东      8
香港      8
陕西      8
辽宁      8
湖北      9
其他     10
四川     13
福建     15
江苏     23
海外     45
浙江     64
广东    107
上海    131
北京    515
Name: 昵称, dtype: int64